Recommendation System implemented Using DeepCoNN architecture with input being user food reviews, food flavanoids data and using tf-idf for embedding.    

In [2]:
!pip install tensorflow==1.14.0

import pandas as pd
import numpy as np
from time import time

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import text_to_word_sequence
from sklearn.feature_extraction.text import TfidfVectorizer


import tensorflow as tf

from keras.models import Model
from keras.callbacks import EarlyStopping, TensorBoard
from keras.layers import Conv1D, MaxPooling1D, Flatten
from keras.layers import Input, Dense , Embedding
from keras.layers.merge import Add, Dot, Concatenate

In [3]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
rawData=pd.read_csv('/content/gdrive/My Drive/Final Year Project/DeepConn/Deep_Learning_Recommender_System/unembedded_grouped_cleaned_data.csv')
rawData.head()

,Unnamed: 0,user_id,food_id,rating,userReviews,foodReviews
0,0,9974,998,5,"['delicious', 'and', 'easy', 'to', 'make']","['delicious', 'and', 'easy', 'to', 'make']"
1,1,10340,998,5,"['feeling', 'healthy', 'great', 'replication',...","['delicious', 'and', 'easy', 'to', 'make']"
2,2,12047,998,5,"['yum', 'will', 'make', 'again']","['delicious', 'and', 'easy', 'to', 'make']"
3,3,13451,998,5,"['turned', 'out', 'great', 'i', 'added', 'pean...","['delicious', 'and', 'easy', 'to', 'make']"
4,8,18226,998,5,"['fantastic', 'tasting', 'completely', 'simple...","['delicious', 'and', 'easy', 'to', 'make']"


In [5]:
rawFlavanoids=pd.read_csv("/content/gdrive/My Drive/Final Year Project/DeepConn/Deep_Learning_Recommender_System/newDatabase.csv")
rawFlavanoids.head()

,Unnamed: 0,id,flavanoids
0,0,1,"Ethyl Lactate|3,4-Dihydroxybenzaldehyde|DL-Liq..."
1,1,2,"AC1LDI49|56424-87-4|3,4-Dihydroxybenzaldehyde|..."
2,2,3,3-Methyl-1-butanol|Thymol|2-Nonanone|Pyrrolidi...
3,3,4,"AC1LDI49|56424-87-4|2-Hexenyl propanoate|3,4-D..."
4,4,5,"3,4-Dihydroxybenzaldehyde|DL-Liquiritigenin|2-..."


In [32]:
flavanoidDB={}
for i in range(rawFlavanoids.shape[0]):
  flavanoidDB[str(rawFlavanoids["id"][i])]=str(rawFlavanoids["flavanoids"][i].split("|"))
print("total flanavnoids={}".format(len(flavanoidDB)))
print(list(flavanoidDB.keys())[:5])

total flanavnoids=1380
['1', '2', '3', '4', '5']


In [0]:
processedData=rawData.copy()
for ele in flavanoidDB.keys():
  try:
    processedData.loc[processedData["food_id"]==ele,"foodReviews"]=flavanoidDB[ele]
  except:
    print("food id {} not found in the dataset".format(ele))


In [25]:
processedData.head()

,Unnamed: 0,user_id,food_id,rating,userReviews,foodReviews
0,0,9974,998,5,"['delicious', 'and', 'easy', 'to', 'make']","['4-Hexen-3-One', 'Mesityl Oxide', '3-Phenylpr..."
1,1,10340,998,5,"['feeling', 'healthy', 'great', 'replication',...","['4-Hexen-3-One', 'Mesityl Oxide', '3-Phenylpr..."
2,2,12047,998,5,"['yum', 'will', 'make', 'again']","['4-Hexen-3-One', 'Mesityl Oxide', '3-Phenylpr..."
3,3,13451,998,5,"['turned', 'out', 'great', 'i', 'added', 'pean...","['4-Hexen-3-One', 'Mesityl Oxide', '3-Phenylpr..."
4,8,18226,998,5,"['fantastic', 'tasting', 'completely', 'simple...","['4-Hexen-3-One', 'Mesityl Oxide', '3-Phenylpr..."


In [0]:
# Train/test split for our model is unique, we need to hold out a
# set of users and movies so that our network never learns those 
testSize = 0.005

# get testSize percentage of users
uniqueUsers = processedData.loc[:, "user_id"].unique()
usersSize = len(uniqueUsers)
test_idx = np.random.choice(usersSize,
                              size=int(usersSize * testSize),
                              replace=False)

# get test users
testUsers = uniqueUsers[test_idx]

# everyone else is a training user
trainUsers = np.delete(uniqueUsers, test_idx)

test = processedData[processedData["user_id"].isin(testUsers)]
train = processedData[processedData["user_id"].isin(trainUsers)]

uniqueTestFood = test["food_id"].unique()

# drop the movies that also appear in our test set. In order to be
# a true train/test split, we are forced to discard some data entirely
train = train.where(np.logical_not(train["food_id"].isin(uniqueTestFood))).dropna()

In [27]:
print("train={} test={}".format(train.shape,test.shape))

train=(16879, 6) test=(272, 6)


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
embeddingSize=241
tfidf=TfidfVectorizer(max_features=241)
tfidf.fit(list(train["userReviews"]))
tfidf_features=tfidf.transform(list(train["userReviews"])).astype("float64")

trainUserReviews=tfidf_features.toarray()
testUserReviews=tfidf.transform(list(test["userReviews"])).toarray()

tfidf=TfidfVectorizer(max_features=len(flavanoidDB))
tfidf.fit(list(train["foodReviews"]))
trainFoodReviews=tfidf.transform(list(train["foodReviews"])).toarray()
testFoodReviews=tfidf.transform(list(test["foodReviews"])).toarray()

In [34]:
print("Embedding Dimensions => userReviews={} || foodReviews={}".format(trainUserReviews.shape,trainFoodReviews.shape))

Embedding Dimensions => userReviews=(16879, 241) || foodReviews=(16879, 1072)


In [0]:
class DeepCoNN():
    def __init__(self,
                 userEmbeddingSize,
                 foodEmbeddingSize,
                 hiddenSize,
                 filters=2,
                 kernelSize=10,
                 strides=6):
        self.userEmbeddingSize = userEmbeddingSize
        self.foodEmbeddingSize=foodEmbeddingSize
        self.hiddenSize = hiddenSize
        self.filters = filters
        self.kernelSize = kernelSize
        self.inputU, self.towerU = self.createDeepCoNnTower(self.userEmbeddingSize)
        self.inputF, self.towerF = self.createDeepCoNnTower(self.foodEmbeddingSize)
        self.joined = Concatenate()([self.towerU, self.towerF])
        self.outNeuron = Dense(1)(self.joined)

    def createDeepCoNnTower(self,embeddingSize):
        inputLayer = Input(shape=(embeddingSize,))
        embeddingLayer=Embedding(embeddingSize,100)(inputLayer)
        tower = Conv1D(filters=self.filters,
                       kernel_size=self.kernelSize,
                       activation="tanh")(embeddingLayer)
        tower = MaxPooling1D()(tower)
        tower = Flatten()(tower)
        tower = Dense(self.hiddenSize, activation="relu")(tower)
        return inputLayer, tower

    def createDeepCoNnDp(self):
        dotproduct = Dot(axes=1)([self.towerU, self.towerF])
        output = Add()([self.outNeuron, dotproduct])
        self.model = Model(inputs=[self.inputU, self.inputF], outputs=[output])
        self.model.compile(optimizer='Adam', loss='mse')
        
    def train(self, trainData,trainUserReviews,trainFoodReviews, batch_size, epochs=1):
        tensorboard = TensorBoard(log_dir="tf_logs/{}".format(pd.Timestamp(int(time()), unit="s")))
        self.createDeepCoNnDp()
        print(self.model.summary())
        
        # userReviews = np.array(trainData.loc[:, "userReviews"])
        # foodReviews = np.array(trainData.loc[:, "foodReviews"])
        userReviews=trainUserReviews
        foodReviews=trainFoodReviews

        self.train_inputs = [userReviews, foodReviews]
        self.train_outputs = trainData.loc[:, "rating"]
        
        self.history = self.model.fit(self.train_inputs,
                                      self.train_outputs,
                                      callbacks=[tensorboard],
                                      validation_split=0.05,
                                      batch_size=batch_size,
                                      epochs=epochs)
        
        

In [36]:
hiddenSize = 64

deepconn = DeepCoNN(trainUserReviews.shape[1],trainFoodReviews.shape[1], hiddenSize)

batch_size = 32
deepconn.train(train,trainUserReviews,trainFoodReviews, batch_size, epochs=10)

deepconn.model.save("cnn.h5")
# print(train_embedded.loc[0]






Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 241)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1072)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 241, 100)     24100       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1072, 100)    107200      input_2[0][0]                    
_______________________________________________________________________________________

In [37]:

testInputs = [testUserReviews, testFoodReviews]
print(testInputs)
# dat = pd.DataFrame(testInputs)
# dat.to_csv("/content/gdrive/My Drive/DeepConn/Deep_Learning_Recommender_System//test_data.csv")

trueRating = np.array(list(test.loc[:, "rating"])).reshape((-1, 1))

predictions = deepconn.model.predict(testInputs)

error = np.square(predictions - trueRating)

print("MSE:", np.average(error))

[array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[0.05802913, 0.        , 0.        , ..., 0.05937492, 0.02074124,
        0.        ],
       [0.04785646, 0.        , 0.        , ..., 0.04896633, 0.01710524,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.01988787,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])]
MSE: 1.1140924191800303
